In [247]:
import pickle
import pandas as pd
import numpy as np
from sklearn.cluster import DBSCAN
from sklearn.cluster import AgglomerativeClustering
from sklearn.preprocessing import normalize
import spacy
from sklearn.feature_extraction import text 

from gensim import corpora, models, similarities, matutils
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import NMF
from sklearn.metrics.pairwise import cosine_similarity

import string
from gensim.utils import lemmatize

from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer
import nltk
import pattern3
import spacy

import gensim

pd.options.display.max_colwidth = 500

Load comments, remove labels

In [248]:
comments_labeled = pickle.load(open('donald_mueller_1.pkl', 'rb'))

In [249]:
comments = [comment[1] for comment in comments_labeled]
labels = [comment[0] for comment in comments_labeled]

Cleaning, removing stop words, lemmatizing

In [250]:
nlp = spacy.load('en', disable=['parser', 'ner'])
additional_stop_words = ['deleted','don','https','pastebin','com','reddit','http','www','ethereum','eth','like','pron']
stop_words = text.ENGLISH_STOP_WORDS.union(additional_stop_words)

def string_clean(comment):
    comment = comment.lower()
    comment = comment.replace('\n',' ') #.replace('*','').replace("\'","").replace('"','').replace('>','').replace('\\',' ')
    comment = " ".join([word.lemma_ for word in nlp(comment)])
    comment = " ".join([word for word in comment.split(" ") if word not in stop_words])
    return comment

corpus = [string_clean(s) for s in comments]
         

In [251]:
len(corpus)

92173

In [252]:
pickle.dump(corpus, open('donald_mueller_1_clean.pkl', 'wb'))

In [253]:
# google_vec_file = '/Users/danielwang/metis/metisgh/NLP_embeddings/GoogleNews-vectors-negative300.bin'

# model = gensim.models.KeyedVectors.load_word2vec_format(google_vec_file, binary=True)

In [257]:
tokenized_docs = [gensim.utils.simple_preprocess(doc) for doc in corpus]

In [258]:
model_size = 100
model = gensim.models.Word2Vec(tokenized_docs, size=model_size, window=5, min_count=3, sg=5)

In [259]:
print(model.wv['trump'])

[ 0.29138407 -0.5523848  -0.1256563   0.8172893  -0.27446973  0.01089713
 -0.5109821   0.2041584  -0.09721227 -0.13040686  0.3644885  -0.11927028
 -0.271812    0.18255568 -0.08609955 -0.15340954 -0.23693712 -0.10003255
  0.43417037 -0.12839112  0.31315213  0.6777304   0.20900205 -0.05283588
  0.23731449 -0.04182166  0.13872667 -0.3250025  -0.44367787 -0.2576545
  0.00699788  0.11366668  0.22139373 -0.24021631 -0.29451898  0.06730222
  0.16875291 -0.055098   -0.12840688 -0.38291875 -0.39816812  0.6314022
 -0.25439233  0.43974394 -0.48401654  0.04912001 -0.1231515   0.38828057
 -0.38397348 -0.20004115 -0.12908308 -0.28720763 -0.03036614 -0.11594658
 -0.4820253   0.01701362  0.06569064  0.16835245 -0.01315062  0.1118971
  0.3928753  -0.09768268 -0.18298775 -0.32465866  0.20008421  0.50045323
  0.22357284 -0.33623973 -0.36764163 -0.37299278 -0.71279836 -0.01438188
  0.23427603 -0.3045042   0.8405727   0.6173249  -0.21606562  0.14275575
  0.26409873  0.03261396  0.01941265  0.1664839   0.11

In [284]:
model.wv.most_similar('donald',topn=30)

[('つthe_donald', 0.710845410823822),
 ('jr', 0.6643936634063721),
 ('つtake', 0.6479265689849854),
 ('falwell', 0.6439195871353149),
 ('energy', 0.6257114410400391),
 ('retweete', 0.6239410638809204),
 ('madman', 0.6215115785598755),
 ('junior', 0.6202978491783142),
 ('winner', 0.6132515668869019),
 ('animate', 0.607268750667572),
 ('af', 0.5997235178947449),
 ('deviantart', 0.5994365215301514),
 ('eld', 0.5950406789779663),
 ('jerry', 0.5941364169120789),
 ('stump', 0.5940910577774048),
 ('benedict', 0.5938334465026855),
 ('rinse', 0.5925987362861633),
 ('ranking', 0.592211127281189),
 ('trump', 0.5887190103530884),
 ('penthouse', 0.5880746841430664),
 ('mount', 0.5869204998016357),
 ('pornstar', 0.5868772268295288),
 ('magazine', 0.5842210054397583),
 ('glover', 0.5836567878723145),
 ('heaven', 0.5821071863174438),
 ('einstein', 0.5791744589805603),
 ('eric', 0.5790146589279175),
 ('cocoa', 0.5782574415206909),
 ('prez', 0.5777601003646851),
 ('bois', 0.5776240825653076)]

In [287]:
model.wv.most_similar(positive=['stormy'], negative=[],topn=30)

[('daniel', 0.9627673625946045),
 ('daniels', 0.9040154218673706),
 ('micheal', 0.862200140953064),
 ('prague', 0.8603132367134094),
 ('blackmail', 0.8547999858856201),
 ('papadopolous', 0.8476603031158447),
 ('confess', 0.8464497327804565),
 ('jared', 0.839411735534668),
 ('avenatti', 0.8289943933486938),
 ('dealing', 0.8276917338371277),
 ('connect', 0.820975661277771),
 ('baron', 0.8170339465141296),
 ('papa', 0.8099651336669922),
 ('hush', 0.8058693408966064),
 ('receipt', 0.8028391003608704),
 ('connection', 0.8026182651519775),
 ('papadopoulo', 0.8017052412033081),
 ('veselnitskaya', 0.8003612756729126),
 ('adviser', 0.7991524934768677),
 ('alleged', 0.7990235090255737),
 ('kilimnik', 0.7983691096305847),
 ('gps', 0.7963666319847107),
 ('dotcom', 0.7963451147079468),
 ('mafia', 0.7954854965209961),
 ('soho', 0.7914476990699768),
 ('stonetear', 0.7882674336433411),
 ('fusion', 0.7882207632064819),
 ('dirt', 0.7873961925506592),
 ('kushner', 0.7852904200553894),
 ('kompromat', 0.78

In [265]:
doc_vectors = []

for doc in tokenized_docs:

    vec = np.zeros(model_size)

    for word in doc:
        if word in model.wv.vocab:
            vec += model.wv[word]
    doc_vectors.append(vec/len(doc))

df = pd.DataFrame(doc_vectors)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in true_divide
  # Remove the CWD from sys.path while we load stuff.


In [266]:
df['subreddit'] = labels

In [267]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score, recall_score, precision_score, log_loss, roc_auc_score, confusion_matrix, roc_curve, precision_recall_curve
from sklearn.model_selection import GridSearchCV

In [268]:
RandomForestClassifier(n_estimators=1000, max_depth=None,random_state=42)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=None,
            oob_score=False, random_state=42, verbose=0, warm_start=False)

In [269]:
df.dropna(axis=0, inplace=True)

In [270]:
X = df.drop(['subreddit'],axis=1)
y = df['subreddit']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=12345)

In [273]:
rf = RandomForestClassifier(n_estimators=50, max_depth=20,random_state=42)

In [272]:
rf.fit(X_train,y_train)

KeyboardInterrupt: 